In [44]:
struct Topography
    f::Function
    params::Tuple
end

"""
Function describing a flat plane
"""
function flat(x)
    0
end

"""
Function describing bump
# Arguments
- `x`: x coordinate
- `w`: scaling of the bump (controls how wide it is)
- `c`: x coordinate for the center of the bump
- `h`: optional arg to scale the height of the bump
"""
function bump(x, w, c, h=1)
    if x <= c-w || x >= c+w
        return 0
    else
        return h * exp(-(w^2/(w^2-(x-c)^2)))
    end
end

"""
Function describing a rectangular mound
# Arguments
- `x`: x coordinate
- `c`: x coordinate for left hand corner of mound
- `d`: height of mound
- `delta`: steepness of mound
- `w`: width of mound
"""
function mound(x, c, h, delta, w)
    h * (1/pi * (atan((x-c)/delta) - atan((x-c-w)/delta)))
end

"""
Function describing a reflected sigmoid function
# Arguments
- `x`: x coordinate
- `b`: precursor film height
- `xf`: center point of transition
- `c`: steepness of transition
"""
function intialFunc(x, b, xf, c)
    b + (1-b)/(1 + exp(1/c * (x - xf)))
end

intialFunc (generic function with 1 method)

In [54]:
"""
Builds the system of ODEs (represented by du)
u = (h_-1, h_0, ..., h_nx, h_nx+1) where h_-1 and h_nx+1 are ghost points
"""
function system!(du, u, p, t)
    D, alpha, dx, topo, params = p

    function phi(i)
        u[i] + topo(dx*(i-2), params...)
    end
    function interp(i)
        1/2 * (u[i]^3 + u[i+1]^3)
    end
    function f1(i)
        1/(dx^2) * (interp(i-1)*(phi(i-1) - phi(i)) + interp(i)*(phi(i+1) - phi(i)))
    end
    function f2(i)
        (1/dx^4) * (interp(i-1)*(phi(i-2) - 3*phi(i-1) + 3*phi(i) - phi(i+1)) + interp(i)*(-phi(i-1) + 3*phi(i) - 3*phi(i+1) + phi(i+2)))
    end
    function f3(i)
        (1/(2*dx)) * (u[i+1]^3 - u[i-1]^3)
    end

    du[1] = 0
    du[2] = 0
    du[end-1] = 0
    du[end] = 0
    for i in 3:length(u)-2
        du[i] = D*cos(alpha)*f1(i) - f2(i) - sin(alpha)*f3(i)
    end
    return du
end

system! (generic function with 1 method)

In [58]:
# Discretization of x-domain
nx = 800
Lx = 40
dx = Lx/nx
x = 0.0:dx:Lx

# Topography definition
bcenter = 20.0
bscale = 5.0
topo = Topography(bump, (bscale, bcenter))
#topo = Topography(flat, ())
s = topo.f.(x, topo.params...)

# Initial condition definition
b = 0.1
xf = 5
steep = 0.5
ic = intialFunc.(x, b, xf, steep)
# Adding ghost points
pushfirst!(ic, ic[1])
push!(ic, ic[end])

# Parameters of the system of ODEs
D = 1
alpha = pi/2

xsym = Symbol.(x)
pushfirst!(xsym, Symbol("ghost1"))
push!(xsym, Symbol("ghost2"))

# Solve the system of ODEs
using DifferentialEquations

p = (D, alpha, dx, topo.f, topo.params)
tspan = (0.0, 30.0); saveEvery = 0.5
f = ODEFunction(system!, syms=xsym)
prob = ODEProblem(f, ic, tspan, p, saveat=saveEvery)
sol = solve(prob, alg=Rodas4())

retcode: Success
Interpolation: 1st order linear
t: 61-element Vector{Float64}:
  0.0
  0.5
  1.0
  1.5
  2.0
  2.5
  3.0
  3.5
  4.0
  4.5
  ⋮
 26.0
 26.5
 27.0
 27.5
 28.0
 28.5
 29.0
 29.5
 30.0
u: 61-element Vector{Vector{Float64}}:
 [0.9999591419181678, 0.9999591419181678, 0.9999548450517846, 0.9999500963277494, 0.999944848234345, 0.9999390482653423, 0.9999326383952405, 0.9999255544994335, 0.999917725713533, 0.9999090737254831  …  0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
 [0.9999591419181704, 0.9999591419182114, 0.9999439616976943, 0.9999162537537887, 0.999878678968597, 0.9998339112347076, 0.999784641215687, 0.9997335792268932, 0.9996834572201768, 0.9996370298545492  …  0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
 [0.9999591419181836, 0.9999591419182324, 0.9999272722028526, 0.9998661568798478, 0.9997784380383716, 0.9996667868147364, 0.9995339136467672, 0.9993825777144978, 0.9992155955601004, 0.9990358488752977  …  0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
 [

In [59]:

using Plots, Dates, LaTeXStrings
Plots.PyPlotBackend()

run_tag = string(Dates.format(now(), "YYYYmmdd-HHMMSS"))
path = "../runs/$run_tag"

plot_font = "Computer Modern"
default(fontfamily=plot_font, grid=false)

dt = floor(Int8, 2/0.5)
temp = sol.u[1:dt:end]
for i in 1:length(temp)
    temp[i] = temp[i][2:end-1]
end

plt = plot(x, s, dpi=200, xlabel=L"Dimensionless length $x$", ylabel=L"Dimensionless height $h$", legend=false, color="black",
        title="Fluid profile at " * L"\delta t = 2" * " intervals", ylims=(0, Inf), xlims=(0, Lx))
for i = 1:length(temp)
    plot!(plt, x, temp[i]+s)
end

anim = @animate for i = 1:size(sol.u)[1]
    plot(x, s, color="black", xlabel=L"Dimensionless length $x$", ylabel=L"Dimensionless height $h$")
    plot!(x, sol.u[i][2:end-1]+s, legend=false, ylims=(0, 2), xlims=(0, Lx))
end

using CSV, DataFrames
mkpath(path)
gif(anim, path * "/flow.gif", fps = 5)
savefig(plt, path * "/plt.png")
df = DataFrame(sol)
#=
CSV format
First Col - Timestamp
Rest of Cols - Data including ghost points
=#
CSV.write(path * "/data.csv", df)

┌ Info: Saved animation to 
│   fn = /Users/bsamineni/Documents/research_projects/acoustics_thin_film/notes/runs/20220224-232502/flow.gif
└ @ Plots /Users/bsamineni/.julia/packages/Plots/LI4FE/src/animation.jl:114


"../runs/20220224-232502/data.csv"